In [3]:
import requests
from typing import Optional, List, Dict, Any
from langchain.llms.base import LLM

class LocalLLM(LLM):
    # ประกาศฟิลด์ให้ Pydantic รู้จัก
    api_url: str
    model_name: str

    @property
    def _llm_type(self) -> str:
        """ชื่อชนิดของ LLM เพื่อ logging ภายใน LangChain"""
        return "local_llm"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model": self.model_name, "api_url": self.api_url}

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> str:
        """เรียก API และคืน response มาเป็นสตริง"""
        res = requests.post(
            f"{self.api_url}/api/generate",
            json={
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,
            },
        )
        res.raise_for_status()
        return res.json().get("response", "")

API_URL = "http://localhost:11434"
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"
llm = LocalLLM(api_url=API_URL, model_name=MODEL_NAME)


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
# Wrap SentenceTransformer ผ่าน LangChain
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

ModuleNotFoundError: Module langchain_community.embeddings not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [3]:
from langchain.document_loaders import TextLoader
import glob

paths = glob.glob("Raw-data-from-TTT\**\*.txt", recursive=True)
documents = []
for p in paths:
    loader = TextLoader(p, encoding="utf-8")
    documents.extend(loader.load())


In [4]:
from langchain.vectorstores import FAISS
# สร้างและเก็บ Document embeddings ลงใน FAISS index
vectorstore = FAISS.from_documents(documents, embeddings)
# บันทึก index ไว้ใช้งานครั้งถัดไป
vectorstore.save_local("faiss_index")

In [5]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Custom prompt template
custom_template = """
บทสนทนาที่ผ่านมา:
{chat_history}

ใช้บริบทต่อไปนี้ในการตอบคำถามที่อยู่ท้ายบท
ผมคือ TTT-Assistant ผู้ช่วย AI ของบริษัท TTT Brothers Co., Ltd.
หากคุณไม่ทราบคำตอบ ให้ตอบเพียงว่าคุณไม่ทราบ อย่าพยายามแต่งคำตอบขึ้นมา

ข้อมูลที่มี:
{context}

คำถาม: {question}

คำตอบ: """

CUSTOM_PROMPT = PromptTemplate(
    template=custom_template,
    input_variables=["chat_history", "context", "question"]
)

# สร้าง Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# สร้าง QA Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT}
)

C:\Users\Arther\AppData\Local\Temp\ipykernel_19752\1769046932.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [7]:
# คำถามตัวอย่าง
question = "แปลว่าอะไรครับ"
# เรียก chain พร้อมส่งคำถาม
result = qa_chain({"question": question})

# ดูประวัติการสนทนา
print("Chat history:\n", memory.load_memory_variables({}))

print(result.get("answer"))

Chat history:
 {'chat_history': [HumanMessage(content='วิสัยทัศน์ของบริษัทคืออะไร?', additional_kwargs={}, response_metadata={}), AIMessage(content='วิสัยทัศน์ของบริษัทคือ "how the world works simplifying"', additional_kwargs={}, response_metadata={}), HumanMessage(content='แปลว่าอะไรครับ', additional_kwargs={}, response_metadata={}), AIMessage(content='คำตอบ: ผมไม่ทราบครับ', additional_kwargs={}, response_metadata={})]}
คำตอบ: ผมไม่ทราบครับ
